In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
# 将建筑和楼层进行合并，形成新的编码


# 1. 读取训练数据（路径请按需修改）
train_df = pd.read_csv('C:/Users/20623/Desktop/project_code/UJIIndoorLoc-PG/UJIIndoorLoc/trainingData.csv')
valid_df = pd.read_csv('C:/Users/20623/Desktop/project_code/UJIIndoorLoc-PG/UJIIndoorLoc/validationData.csv')
train_df_noisy1 = pd.read_csv('../../data/train_noisy1.csv')

# train_df = pd.concat([train_df, train_df_noisy1], ignore_index=True)
# total_df = pd.concat([train_df, valid_df], ignore_index=True)

# 2. 创建联合标签列（如 "2_3" 表示 BUILDINGID=2 且 FLOOR=3）
train_df['location_label'] = train_df['BUILDINGID'].astype(str) + '_' + train_df['FLOOR'].astype(str)
valid_df['location_label'] = valid_df['BUILDINGID'].astype(str) + '_' + valid_df['FLOOR'].astype(str)
# total_df['location_label'] = total_df['BUILDINGID'].astype(str) + '_' + total_df['FLOOR'].astype(str)

# 3. 将联合标签进行整数编码
label_encoder = LabelEncoder()
train_df['location_label_encoded'] = label_encoder.fit_transform(train_df['location_label'])
valid_df['location_label_encoded'] = label_encoder.fit_transform(valid_df['location_label'])
# total_df['location_label_encoded'] = label_encoder.fit_transform(valid_df['location_label'])
total_df = pd.concat([train_df, valid_df])
# 4. 保存为 CSV 文件
# train_df.to_csv('./data/processed_train.csv', index=False)
# valid_df.to_csv('./data/processed_valid.csv', index=False)

# print("✅ 文件已成功保存为 'processed_train.csv'")
# print("✅ 文件已成功保存为 'processed_validat.csv'")

In [3]:
# 取特征和标签
# 训练集
training_data = train_df[train_df.columns[:520]].to_numpy()
training_floors = train_df['location_label_encoded'].to_numpy() # FLOOR LABELS
# 验证集
valid_data = valid_df[valid_df.columns[:520]].to_numpy()
valid_floors = valid_df['location_label_encoded'].to_numpy() # FLOOR LABELS
# 数据总和
total_data = total_df[total_df.columns[:520]].to_numpy()
total_floors = total_df['location_label_encoded'].to_numpy() # FLOOR LABELS

In [4]:
# 数据归一化处理
from Algorithms.utill.data_standar import normalize_rssi, normalize_coords, normalize_test_or_valid_data
# 数据标准化,从总数居获取最大值最小值
X_totalCo_cnn, X_min, X_max = normalize_rssi(total_data)
print(X_min, X_max)
# 训练集特征标准化
training_data = normalize_test_or_valid_data(X_min, X_max, training_data)
# 验证集特征标准化
valid_data = normalize_test_or_valid_data(X_min, X_max, valid_data)
# 全集
total_data = normalize_test_or_valid_data(X_min, X_max, total_data)

-104 100


# 填补23*23进行CNN

In [5]:
# Augmenting data shape to fit 23x23
training_data_aug = np.empty((len(training_data),529))
for x in range(len(training_data)):
    training_data_aug[x] = np.concatenate((training_data[x], np.full(shape=9,fill_value=1)))
training_data_aug.shape
# training_data_aug = training_data

# Augmenting data shape to fit 23x23
valid_data_aug = np.empty((len(valid_data),529))
for x in range(len(valid_data)):
    valid_data_aug[x] = np.concatenate((valid_data[x], np.full(shape=9,fill_value=1)))
# valid_data_aug.shape
# training_data_aug = training_data
total_data_aug = np.empty((len(total_data),529))
for x in range(len(total_data)):
    total_data_aug[x] = np.concatenate((total_data[x], np.full(shape=9,fill_value=1)))
# valid_data_aug.shape

In [6]:
from sklearn.model_selection import train_test_split

# total_space_ids = total_df['SPACEID'].values
# X_temp, X_testF_cnn, y_temp, y_testF_cnn, stratify_temp, stratify_test= train_test_split(total_data_aug.reshape(len(total_data_aug),23,23), total_floors, test_size=0.2, random_state=2812,stratify=total_space_ids)
# 
# X_trainF_cnn, X_valid_F_cnn, y_trainF_cnn, y_valid_F_cnn = train_test_split(
#     X_temp,
#     y_temp,
#     test_size=1/8,   # 因为 90% * (2/9) ≈ 20%
#     random_state=2812,
#     stratify=stratify_temp
# )

# 准备数据
X = total_data_aug.reshape(len(total_data_aug), 23, 23)
y = total_floors
z = total_df['SPACEID'].to_numpy()  # 分层标签

assert len(X) == len(y) == len(z), (len(X), len(y), len(z))

# 第一步：先切 20% 测试集
X_temp, X_testF_cnn, y_temp, y_testF_cnn, stratify_temp, stratify_test = train_test_split(
    X, y, z,
    test_size=0.1,               # 20% → 测试
    random_state=2812,
    stratify=z
)

In [7]:
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import os
from Algorithms.CNN import CNNClassifier

# ===== 0) 固定测试集 =====
X = total_data_aug.reshape(len(total_data_aug), 23, 23)
y = total_floors
z = total_df['SPACEID'].to_numpy()  # 分层标签（用空间/房间分层更稳）

X_trainval, X_testF_cnn, y_trainval, y_testF_cnn, z_trainval, z_test = train_test_split(
    X, y, z, test_size=0.2, random_state=2812, stratify=z
)

# ===== 1) K 折交叉验证（在 80% 训练验证集上）=====
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2812)

dropout_list = [0.3]
lr_list = [0.01]
kernel_sizes = [3]
strides = [1]
weight_decays = [0.001]

save_dir = "./model"
os.makedirs(save_dir, exist_ok=True)

fold_val_scores = []
fold_model_paths = []

model_order = 82  # 你原来在用的编号

for fold, (tr_idx, va_idx) in enumerate(skf.split(X_trainval, z_trainval), start=1):
    X_trainF_cnn, X_valid_F_cnn = X_trainval[tr_idx], X_trainval[va_idx]
    y_trainF_cnn, y_valid_F_cnn = y_trainval[tr_idx], y_trainval[va_idx]

    print(f"\n===== Fold {fold}/{n_splits} =====")
    print("Train:", X_trainF_cnn.shape, " Valid:", X_valid_F_cnn.shape)

    best_val_acc_this_fold = -1.0
    best_model_path_this_fold = None

    # 网格可继续外套超参；示例沿用你当前单点配置
    for val in dropout_list:
        for lr in lr_list:
            for kernel_size in kernel_sizes:
                for stride in strides:
                    for weight_decay in weight_decays:
                        padding = 0
                        sava_model_name = os.path.join(save_dir, f"{model_order}_fold{fold}.pth")

                        print(f"(Fold {fold}) Train cfg: dropout={val}, lr={lr}, k={kernel_size}, s={stride}, wd={weight_decay}")
                        cnn = CNNClassifier(
                            n_classes=13, dropout=val,
                            kernel_size=kernel_size, stride=stride, padding=padding
                        )
                        # 用“当前折”的 train/valid 做训练与早停
                        cnn.fit(X_trainF_cnn, y_trainF_cnn, X_valid_F_cnn, y_valid_F_cnn)
                        # 训练并保存当前配置的最优权重
                        val_acc = cnn.train_model(
                            num_epochs=50, eval_train=True, lr=lr, batch_size=64, patience=7,
                            sava_model_name=sava_model_name, kernel_size=kernel_size, stride=stride,
                            dropout=val, weight_decay=weight_decay
                        )
                        print(f"(Fold {fold}) Val Acc: {val_acc:.4f}")
                        model_order += 1

                        # 记录本折最佳
                        if val_acc is not None and val_acc > best_val_acc_this_fold:
                            best_val_acc_this_fold = val_acc
                            best_model_path_this_fold = sava_model_name

    print(f"[Fold {fold}] Best Val Acc = {best_val_acc_this_fold:.4f} @ {best_model_path_this_fold}")
    fold_val_scores.append(best_val_acc_this_fold)
    fold_model_paths.append(best_model_path_this_fold)

# ===== 2) 选择“最佳折”模型，在测试集上评估 =====
best_fold_idx = int(np.argmax(fold_val_scores))
best_path = fold_model_paths[best_fold_idx]
print(f"\n>>> Best fold = {best_fold_idx+1}, Val Acc = {fold_val_scores[best_fold_idx]:.4f}")
print(f">>> Loading best fold model: {best_path}")

model = CNNClassifier(n_classes=13, kernel_size=3, stride=1, padding=0, dropout=0.3)
model.load_model(best_path)
# 仅为 dataloader 占位；不使用 train
model.X_train = np.zeros_like(X_testF_cnn)
model.Y_train = np.zeros_like(y_testF_cnn)
model.X_test = X_testF_cnn
model.Y_test = y_testF_cnn
model.create_loaders(batch_size=64)
test_acc, test_pred = model.test(model.loaders['test'])
print(f"[Best-fold model] Final Test Acc: {test_acc:.4f}")


===== Fold 1/5 =====
Train: (13470, 23, 23)  Valid: (3368, 23, 23)
(Fold 1) Train cfg: dropout=0.3, lr=0.01, k=3, s=1, wd=0.001
Epoch [1/50], Step [211/211], Loss: 2.6117
Training Accuracy: 12.887899398803711
Test Accuracy: 13.539192199707031
-> Model improved. Saving 
------------------------------
Epoch [2/50], Step [211/211], Loss: 2.6130
Training Accuracy: 12.887899398803711
Test Accuracy: 13.539192199707031
-> No improvement. Patience: 1/7
------------------------------
Epoch [3/50], Step [211/211], Loss: 2.6120
Training Accuracy: 12.887899398803711
Test Accuracy: 13.539192199707031
-> No improvement. Patience: 2/7
------------------------------
Epoch [4/50], Step [211/211], Loss: 2.6110
Training Accuracy: 12.887899398803711
Test Accuracy: 13.539192199707031
-> No improvement. Patience: 3/7
------------------------------
Epoch [5/50], Step [211/211], Loss: 0.7727
Training Accuracy: 71.21009826660156
Test Accuracy: 70.7244644165039
-> Model improved. Saving 
----------------------

KeyboardInterrupt: 

## CNN Classifier

In [61]:

# Floor Classifier
# dropout = [0.3]
# lrs = [0.01]
# kernel_sizes = [3]
# strides = [1]
# model_order = 82
# weight_decays = [0.001]
# for val in dropout:
#     for lr in lrs:
#         for kernel_size in kernel_sizes:
#             for stride in strides:
#                 for weight_decay in weight_decays:
#                     # padding = compute_padding(kernel_size, stride)
#                     padding = 0
#                     print(f'(Dropout: {val})')
#                     cnn = CNNClassifier(n_classes=13, dropout=val, kernel_size=kernel_size, stride=stride, padding=padding) 
#                     cnn.fit(X_trainF_cnn, y_trainF_cnn, X_testF_cnn, y_testF_cnn)
#                     cnn.train_model(num_epochs=50,eval_train=True, lr=lr, batch_size=64, patience = 7, sava_model_name="./model/"+str(model_order)+"test.pth", kernel_size = kernel_size, stride = stride, dropout=val, weight_decay=weight_decay)
#                     model_order += 1

(Dropout: 0.3)
Epoch [1/50], Step [231/231], Loss: 0.9279
Training Accuracy: 66.3680191040039
Test Accuracy: 66.60332489013672
-> Model improved. Saving 
------------------------------
Epoch [2/50], Step [231/231], Loss: 0.4732
Training Accuracy: 84.74173736572266
Test Accuracy: 84.2992935180664
-> Model improved. Saving 
------------------------------
Epoch [3/50], Step [231/231], Loss: 0.4313
Training Accuracy: 85.05396270751953
Test Accuracy: 85.24940490722656
-> Model improved. Saving 
------------------------------
Epoch [4/50], Step [231/231], Loss: 0.4004
Training Accuracy: 86.3707275390625
Test Accuracy: 85.24940490722656
-> No improvement. Patience: 1/7
------------------------------
Epoch [5/50], Step [231/231], Loss: 0.4124
Training Accuracy: 88.73956298828125
Test Accuracy: 87.9334945678711
-> Model improved. Saving 
------------------------------
Epoch [6/50], Step [231/231], Loss: 0.3215
Training Accuracy: 89.0382080078125
Test Accuracy: 88.43230438232422
-> Model improve

In [62]:
# # 验证模型效果
# from Algorithms.CNN import CNNRegressor
# import numpy as np
# # 假设你的模型是 CNNRegressor，有两个目标值
# dropout = [0.3]
# lrs = [0.001]
# kernel_sizes = [3]
# strides = [1]
# model_order = 82
# weight_decays = [0.0001]
# # 加载保存的权重
# for val in dropout:
#     for lr in lrs:
#         for kernel_size in kernel_sizes:
#             for stride in strides:
#                 for weight_decay in weight_decays:
#                     sava_model_name="./model/"+str(model_order)+"test.pth"
#                     padding = 0
#                     model = CNNClassifier(n_classes=13, kernel_size=kernel_size, stride=stride, padding=padding, dropout=val)
#                     model.load_model(sava_model_name)
#                     model.X_train = np.zeros_like(X_valid_F_cnn)  # 随便补一个假的训练集占位，不会用到
#                     model.Y_train = np.zeros_like(y_valid_F_cnn)
#                     # 注意：还需要加载相应的数据
#                     model.X_test = X_valid_F_cnn
#                     model.Y_test = y_valid_F_cnn
#                     # model.X_train = None
#                     # model.y_train = None
#                     model.create_loaders(batch_size=64)
#                     correct,_ = model.test(model.loaders['test'])
#     
#                     print(f"valid collection: {correct} ")
#                     model_order += 1

valid collection: 96.86460876464844 
